In [35]:
import pandas as pd
import json
import ast

In [36]:
file = "/Users/laurasuemitsu/Desktop/DATA_FT_May_2025/Final project/Tourism datasets Kaggle/Worldwide_Travel_Cities_Dataset_(Ratings_Climate).csv"

In [37]:
travel_df = pd.read_csv(file)

## *Wrangling duration:*

In [38]:
def wrangling_duration(df):
    df["avg_temp_monthly"] = df["avg_temp_monthly"].apply(json.loads)
    temp_monthly = pd.json_normalize(df["avg_temp_monthly"])
    df = pd.concat([df.drop(columns=["avg_temp_monthly"]), temp_monthly], axis=1)
    
    for n in range(1,13):
        df.drop(columns=[(str(n)) + ".min", (str(n)) + ".max"], inplace=True)

    df["ideal_durations"] = df["ideal_durations"].apply(ast.literal_eval)

    df["ideal_durations"] = df["ideal_durations"].apply(lambda lst: ' '.join(item.strip() for item in lst))

    df["ideal_durations"] = [duration.lower() for duration in df["ideal_durations"].values]
    return df

In [39]:
travel_df = wrangling_duration(travel_df)

## *Wrangling budget*

In [40]:
def wrangling_budget(df):
    df["budget_level"] = [budget.lower() for budget in df["budget_level"].values]
    df["budget_level"] = df["budget_level"].replace({"budget": "economic"})
    return df

In [41]:
travel_df = wrangling_budget(travel_df)

## *Removing months from the final df:*

In [42]:
try:
    month = int(input("In which month of the year do you want to go on your trip? Please enter a number between 1 and 12."))
    if month in range(1,13):
        print(month)
    else:
        while month not in range(1,13):
            month = int(input("Please enter a number between 1 and 12."))
        else:
            print(month)
except ValueError:
    month = int(input("Please enter a valid number between 1 and 12."))
    while month not in range(1,13):
        month = int(input("Please enter a number between 1 and 12."))
    else:
        print(month)

8


In [43]:
def remove_months(df):
    months = list(range(1,13))
    no_month_trip = [x for x in months if x != month]
    df = df.drop(columns=[str(x)+".avg" for x in no_month_trip])
    return df

In [44]:
travel_df = remove_months(travel_df)

## *Running the functions:*

In [45]:
travel_df = pd.read_csv(file)

In [46]:
df_airports = pd.read_csv("/Users/laurasuemitsu/Desktop/DATA_FT_May_2025/Final project/Notebooks/airports_codes.csv")

In [47]:
def wrangling(df):
    df["avg_temp_monthly"] = df["avg_temp_monthly"].apply(json.loads)
    temp_monthly = pd.json_normalize(df["avg_temp_monthly"])
    df = pd.concat([df.drop(columns=["avg_temp_monthly"]), temp_monthly], axis=1)
    
    for n in range(1,13):
        df.drop(columns=[(str(n)) + ".min", (str(n)) + ".max"], inplace=True)

    df["ideal_durations"] = df["ideal_durations"].apply(ast.literal_eval)
    df["ideal_durations"] = df["ideal_durations"].apply(lambda lst: ' '.join(item.strip() for item in lst))
    df["ideal_durations"] = [duration.lower() for duration in df["ideal_durations"].values]

    df["budget_level"] = [budget.lower() for budget in df["budget_level"].values]
    df["budget_level"] = df["budget_level"].replace({"budget": "economic"})
    
    return df

In [48]:
travel_df = wrangling(travel_df)

In [49]:
def remove_months(df):
    months = list(range(1,13))
    no_month_trip = [x for x in months if x != month]
    df = df.drop(columns=[str(x)+".avg" for x in no_month_trip])
    return df

In [50]:
def filter_df(travel_df):
    """This function will filter the dataset containing the information of 560 cities around the world to provide the user with a ranking based on their preferences."""

    # Valid inputs definition
    valid_durations = {"weekend", "short trip", "one week", "long trip"}
    valid_budgets = {"luxury", "mid-range", "economic"}
    valid_interests = ["urban", "culture", "adventure", "nature", "beaches", "nightlife", "cuisine", "wellness", "seclusion"]
    
    # Ideal duration input
    ideal_duration = input("How long do you want your trip to be: weekend, short trip, one week, or long trip? ").lower()
    while ideal_duration not in valid_durations:
        ideal_duration = input("Please enter one of the following values: weekend, short trip, one week, or long trip. ").lower()

    # Budget level input
    budget_level = input("What is your budget for your trip: luxury, mid-range, or economic? ").lower()
    while budget_level not in valid_budgets:
        budget_level = input("Please enter one of the following values: luxury, mid-range, or economic. ").lower()

    # Month of the trip input
    month = None
    while month not in range(1, 13):
        try:
            month = int(input("In which month of the year do you want to go on your trip? Please enter a number between 1 and 12. "))
        except ValueError:
            print("Please enter a valid number between 1 and 12.")

    # User main interest input
    while True:
        interests_input = input("Select up to three interests (if more than one, comma separated) among: culture, adventure, nature, beaches, nightlife, cuisine, wellness, urban, and seclusion: ").lower().split(", ")
        if all(interest in valid_interests for interest in interests_input) and len(interests_input) <= 3:
            break
        else:
            print("Invalid input. Please make sure to select up to three interests from the given options.")

    # Creating the filtered DataFrame
    try:
        filtered_df = travel_df[(travel_df["ideal_durations"].str.contains(ideal_duration, case=False)) & (travel_df["budget_level"] == budget_level)]

        # Filter out months if function exists
        if 'remove_months' in globals():
            filtered_df = remove_months(filtered_df)

        # Sort by interest columns; make sure column names exist in the DataFrame
        pd.set_option('display.max_colwidth', None)
        interest_columns = [col for col in interests_input if col in travel_df.columns]
        filtered_df = filtered_df.sort_values(by=interest_columns, ascending=False)
        
        # Drop columns
        filtered_df = filtered_df.drop(columns=["id", "region", "latitude", "longitude", "ideal_durations", "budget_level", "culture", "adventure",	"nature", "beaches", "nightlife", "cuisine", "wellness", "urban", "seclusion"], axis=1)
        
        # Show only top 5 cities
        filtered_df = filtered_df[:5]

        # Renaming average temperature month
        column_to_rename = f"{month}.avg"
    
        if column_to_rename not in filtered_df.columns:
            print(f"Column '{column_to_rename}' does not exist.")
        else:
            new_column_name = "avg_temperature(ºC)"
            filtered_df = filtered_df.rename(columns={column_to_rename: new_column_name})

        message = "Here are the top 5 cities where you could go according to your preferences: "

    except Exception as error:
        print(f"Error in filtering: {error}")

    return message, filtered_df

In [51]:
# Next steps for the 'app':

# >> OK 1. function to ask for the user input regarding user's city choice, where he'll be travelling from and on which day of the month
# >> OK 2. function to run Google Flights API and get flights -- major challenge: how to make the link between city name and airport code? --> OK
# 3. improve function API to handle when flights are not available from the dep city to the arrival city
# 4. function to get hotels if enough time

In [52]:
def get_airport_code(city_name, df):
    # Locate the city in the DataFrame
    result = df.loc[df['city'] == city_name, 'IATA']
    return result.iloc[0] if not result.empty else None

In [53]:
df_airports = pd.read_csv("/Users/laurasuemitsu/Desktop/DATA_FT_May_2025/Final project/Notebooks/airports_codes.csv")

In [54]:
def departuring_details():
    while True:
        departure_city = input("Enter the departure city: ")
        departure_code = get_airport_code(departure_city, df_airports)
        if departure_code:
            break
        else:
            departure_city = print("No airport found in our database. Please enter another city. ")

    while True:
        arrival_city = input("Enter the arrival city: ")
        arrival_code = get_airport_code(arrival_city, df_airports)
        if arrival_code:
            break
        else:
            arrival_city = print("No airport found in our database. Please enter another city. ")

    date_input = input("Enter the departure date (YYYY-MM-DD): ")

    return departure_code, arrival_code, date_input


In [55]:
message, cities_df = filter_df(travel_df)
print(message)
display(cities_df)
departure_code, arrival_code, date_input = departuring_details()
print(departure_code, " - ", arrival_code, " - ", date_input)

Here are the top 5 cities where you could go according to your preferences: 


,city,country,short_description,avg_temperature(ºC)
153,Catania,Italy,"Sun-soaked streets filled with the aroma of fresh seafood, lively markets, and the dramatic backdrop of a smoking volcano create a unique atmosphere.",27.1
110,Taipei,Taiwan,"Bustling night markets filled with tantalizing aromas, serene temples, and lush green hills create a vibrant and welcoming atmosphere.",30.2
44,Oaxaca City,Mexico,Colorful streets filled with vibrant markets and the aroma of rich mole sauces create a warm and inviting cultural tapestry.,21.4
45,San Sebastián,Spain,"Golden beaches meet vibrant pintxo bars and lush green hills, creating a lively yet laid-back atmosphere perfect for unwinding and exploring.",19.9
312,Puglia,Italy,"Sun-drenched coastlines, charming whitewashed villages, and the aroma of olive groves create a serene and inviting atmosphere.",26.4


TNT  -  CDG  -  2025-07-20


## *Implementing API*

In [56]:
import requests
import json

In [57]:
def run_API (departure_code, arrival_code, date_input):
    url = "https://google-flights2.p.rapidapi.com/api/v1/searchFlights"
    headers = {"X-RapidAPI-Key": "ba7cdcaca5mshe733bf910269aafp1b245cjsnef6eeb88a0b5",
               "X-RapidAPI-Host": "google-flights2.p.rapidapi.com"
    }

    querystring = {"departure_id": departure_code,
    "arrival_id": arrival_code,
    "outbound_date": date_input,
    }

    response = requests.get(url, headers=headers, params=querystring)

    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        data = response.json()

        # Optionally process or return the data
        return data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")

    # Return None if there is an error
    return None

In [58]:
def check_airport(departure_code, arrival_code, date_input):
    if departure_code and arrival_code:
        result = run_API(departure_code, arrival_code, date_input)
        return result
    else:
        print("No airport found in our database. Please restart the app.")

In [59]:
def normalize(row):
    flight_info = pd.json_normalize(row)
    return flight_info

In [60]:
def get_df(API_info):
    df_0 = pd.json_normalize(API_info)
    if len(df_0['data.itineraries.topFlights'][0]) < 1:
        print("There are no flights available for these cities and date.")
    else:
        df_0 = pd.json_normalize(df_0['data.itineraries.topFlights'][0], sep='_')
        df_1 = normalize(df_0["flights"])
        df_2 = normalize(df_1[0])
        df_3 = pd.concat([df_0, df_2], axis=1)
        df_4 = df_3.drop(columns=["flights", "airline_logo", "duration_raw", "booking_token", "extensions", "departure_airport.time", "arrival_airport.time", "self_transfer", "layovers", "delay_values", "delay_text", "duration.raw", "duration.text", "carbon_emissions_difference_percent", "carbon_emissions_CO2e", "carbon_emissions_typical_for_this_route", "carbon_emissions_higher", "carbon_emissions_difference_percent", "carbon_emissions_CO2e", "carbon_emissions_typical_for_this_route", "carbon_emissions_higher"])
        return df_4

In [61]:
def choose_flight():
    flight = input("Which flight would you like to choose? Please enter its index (number on the first column of the table): ")
    while flight.isnumeric == True:
        return flight
    else:
        flight = input("Please, enter the number corresponding to your chosen flight: ")
    return flight

In [66]:
departure_code, arrival_code, date = departuring_details()
API_info = check_airport(departure_code, arrival_code, date)
flights = get_df(API_info)
if flights is None or flights.empty:
    print("Please restart the app.")
else:
    print("Here are the top available flights for your trip:")
    display(flights)
    choose_flight()

HTTP error occurred: 429 Client Error: Too Many Requests for url: https://google-flights2.p.rapidapi.com/api/v1/searchFlights?departure_id=TLS&arrival_id=OPO&outbound_date=2025-07-20


NotImplementedError: 